# Module 6 Homework (Cross validation)

Using the ccdefault dataset, with 90% for training and 10% for test (stratified sampling) and the decision tree model that you did in Module 2:

# Part 1: Random test train splits

#Run in-sample and out-of-sample accuracy scores for 10 different samples by changing random_state from 1 to 10 in sequence. 
Display the individual scores, then calculate the mean and standard deviation on the set of scores.  Report in a table format.

In [1]:
import pandas as pd
from pandas import DataFrame as df
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import time
from sklearn.preprocessing import StandardScaler

In [2]:
ccdefault = pd.read_csv("D:/UIUC_Course/Machine Learning/Module6//ccdefault.csv",header = None)
ccdefault.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT
1,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
2,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0


In [3]:
#random test
dt = DecisionTreeClassifier(criterion='gini', max_depth=4, random_state=1)
sc = StandardScaler()
#dt = DecisionTreeClassifier()
default_data=ccdefault.iloc[1:,1:24].values
default_target=ccdefault.iloc[1:,24].values

In [4]:
X=default_data
y=default_target
in_sample_accuracy_set = []
out_sample_accuracy_set = []
timestamp_1 = time.time()
for r in range(1,11): 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state = r, stratify=y)
    sc.fit(X_train)
    X_train_std = sc.transform(X_train)
    X_test_std = sc.transform(X_test)
    dt.fit(X_train_std, y_train)
    y_train_pred  = dt.predict(X_train_std)
    y_test_pred = dt.predict(X_test_std) 
    in_sample_accuracy = accuracy_score(y_train, y_train_pred)
    out_sample_accuracy = accuracy_score(y_test, y_test_pred) 
    in_sample_accuracy_set.append(in_sample_accuracy)
    out_sample_accuracy_set.append(out_sample_accuracy)
timestamp_2 = time.time()
random_state_time = timestamp_2 - timestamp_1
in_sample_accuracy_mean = np.mean(in_sample_accuracy_set)
in_sample_accuracy_std = np.std(in_sample_accuracy_set)
out_sample_accuracy_mean = np.mean(out_sample_accuracy_set)
out_sample_accuracy_std = np.std(out_sample_accuracy_set)
    

In [5]:
print ("10 times random state running time is " + str(random_state_time))

10 times random state running time is 18.371050596237183


In [6]:
in_sample_accuracy_set

[0.8225555555555556,
 0.8225925925925925,
 0.8241111111111111,
 0.8238518518518518,
 0.8231481481481482,
 0.8238888888888889,
 0.8232592592592592,
 0.8237777777777778,
 0.8238518518518518,
 0.8237777777777778]

In [7]:
accuracy_table=df({"Random_State":range(1,11),
        "In_Sample_Accuracy_Scores":in_sample_accuracy_set,
        "Out_of_Sample_Accuracy_Scores":out_sample_accuracy_set})
accuracy_table

,Random_State,In_Sample_Accuracy_Scores,Out_of_Sample_Accuracy_Scores
0,1,0.822556,0.828333
1,2,0.822593,0.824000
2,3,0.824111,0.817333
3,4,0.823852,0.820333
4,5,0.823148,0.818000
5,6,0.823889,0.819000
6,7,0.823259,0.825333
7,8,0.823778,0.816667
8,9,0.823852,0.817333
9,10,0.823778,0.820667


In [8]:
accuracy_stat_table = df(np.arange(1,5).reshape((2, 2)), index=["Accuracy_mean", "Accuracy_std"], columns=["In_Sample", "Out_Sample"])
accuracy_stat_table['In_Sample']=in_sample_accuracy_mean,in_sample_accuracy_std
accuracy_stat_table['Out_Sample']=out_sample_accuracy_mean,out_sample_accuracy_std
accuracy_stat_table

,In_Sample,Out_Sample
Accuracy_mean,0.823481,0.82070
Accuracy_std,0.000531,0.00374


In [9]:
diff = out_sample_accuracy_mean - in_sample_accuracy_mean
print ("The difference bewteen Out_sample and In_sample scores mean is " + str(diff))

The difference bewteen Out_sample and In_sample scores mean is -0.002781481481481607


# Part 2: Cross validation

Now rerun your model using cross_val_scores with k-fold CV (k=10).  
Report the individual fold accuracy scores, the mean CV score and the standard deviation of the fold scores.  Now run the out-of-sample accuracy score.  Report in a table format.

In [11]:
from sklearn.model_selection import cross_val_score

In [12]:
timestamp_3 = time.time()
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1, random_state=1)
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)
cv_scores = cross_val_score(dt, X_train_std, y_train, cv=10,n_jobs=-1)
timestamp_4 = time.time()
cv_running_time=timestamp_4 - timestamp_3

In [13]:
print ("10 folds running time is " + str(cv_running_time))

10 folds running time is 5.544317007064819


In [14]:
cv_scores

array([0.81222222, 0.8162963 , 0.82185185, 0.82074074, 0.81407407,
       0.81777778, 0.82222222, 0.82259259, 0.82518519, 0.82740741])

In [15]:
accuracy_table=df({"K-fold":range(1,11),
        "K-fold Scores":cv_scores,})
accuracy_table

,K-fold,K-fold Scores
0,1,0.812222
1,2,0.816296
2,3,0.821852
3,4,0.820741
4,5,0.814074
5,6,0.817778
6,7,0.822222
7,8,0.822593
8,9,0.825185
9,10,0.827407


In [16]:
cv_mean=np.mean(cv_scores)
cv_std=np.std(cv_scores)

In [17]:
cv_stat_table = df(np.arange(1,3).reshape((2, 1)), index=["mean", "std"], columns=["10-fold Scores"])
cv_stat_table

,10-fold Scores
mean,1
std,2


In [18]:
cv_stat_table['10-fold Scores']=cv_mean,cv_std
cv_stat_table

,10-fold Scores
mean,0.820037
std,0.004601


In [19]:
y_test_pred = dt.predict(X_test_std)
out_sample_scores = accuracy_score(y_test, y_test_pred)
print('Out_of_sample_accuracy:' + str(out_sample_scores))
diff2 = out_sample_scores - in_sample_accuracy_mean
print ("The difference bewteen Out_sample and In_sample scores mean is " + str(diff2))

Out_of_sample_accuracy:0.8256666666666667
The difference bewteen Out_sample and In_sample scores mean is 0.002185185185185179


In [20]:
print("My name is Jialing Zhu")
print("My NetID is: jz72")
print("I hereby certify that I have read the University policy on Academic Integrity and that I am not in violation.")

My name is Jialing Zhu
My NetID is: jz72
I hereby certify that I have read the University policy on Academic Integrity and that I am not in violation.
